# NIMS-KISTI-NVIDIA Hackathon 

# Problem Statement 



## Agenda 
 - Introduction to FourCastNet
 - Configure FourCastNet
 - Typoon Dataset (JMA best track) 
 - ECMWF ERA5 dataset(CDS API)
 - custom interval 
 - inference 
 - post processing 
 




# ECMWF ERA5 dataset 

 - [URL](https://climate.copernicus.eu/climate-reanalysis)
 

<img src="https://www.ecmwf.int/sites/default/files/medialibrary/2019-01/C3S_annual_anomaly_Global_ei_2T_2018_logo_690px.jpg">

## download dataset with CDS API 

### FourCastNet modeled variables
<table align="left" border="1">
  <tr>
    <th>Vertical Level</th>
    <th>Variable</th>
  </tr>
  <tr>
    <td>Surface</td>
    <td>U10, V10, T2M, SP, MSLP</td>
  </tr>
  <tr>
    <td>1000 hPa</td>
    <td>U, V, Z</td>
  </tr>
  <tr>
    <td>850 hPa</td>
    <td>T, U, V, Z, RH</td>
  </tr>
  <tr>
    <td>500 hPa</td>
    <td>T, U, V, Z, RH</td>
  </tr>
  <tr>
    <td>50 hPa</td>
    <td>Z</td>
  </tr>
  <tr>
    <td>Integrated</td>
    <td>TCWV</td>
  </tr>
</table>



In [ ]:
!ls -alh custom_interval

In [ ]:
file_path = 'custom_interval/2214_2022091212_2022092000_sl.nc' 
#file_path = 'custom_interval/2211_2022082718_2022090900_sl.nc'

# visualize ECMWF ERA5 dataset ( temperature, pressure, wind)

## get value  from netcdf4 files

In [ ]:
from netCDF4 import Dataset, num2date
import xarray as xr

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from IPython.display import Image as ImageDisplay, display
import warnings    
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

    
from netCDF4 import Dataset, num2date

def extract_netcdf_info(file_path):
    # Open the netCDF file
    dataset = Dataset(file_path)

    # Get and print dates
    time_var = dataset.variables["time"]
    dates = num2date(time_var[:], units=time_var.units)
    #print(f"Dates: {dates}")

    # Get and print variable list
    var_list = list(dataset.variables.keys())
    print(f"Variable List: {var_list}")
    return dates, var_list
   
def get_variable_data(file_path, target_variable):
    # Open the netCDF file
    dataset = Dataset(file_path)

    # Initialize the list to store variable data
    var_data = []

    # Extract time, latitude, and longitude by default

    
    # Extract the target variable
    var_data =dataset.variables[target_variable][:]
    
    # Close the dataset
    dataset.close()

    return var_data





## visualize ECMWF ERA5  temperature (Kelvin)

In [ ]:

import matplotlib.pyplot as plt
import numpy as np
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

def visualize_temperature_animation(var_data, projection='3d'):
    var_data = np.flip(var_data, axis=1)
    central_longitude = 130
    central_latitude = 10.0
    longitude = np.linspace(0, 360, var_data.shape[2])

    if projection == '3d':
        fig = plt.figure(figsize=(10, 6))
        ax = plt.axes(projection=ccrs.Orthographic(central_longitude=central_longitude, central_latitude=central_latitude))
    elif projection == '2d':
        fig = plt.figure(figsize=(10, 5))
        ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=central_longitude))
 
    else:
        raise ValueError("Invalid projection value. Choose either '2d' or '3d'.")

    coastline = cfeature.COASTLINE.with_scale('110m')

    # Create grid coordinates for pcolormesh
    lon_grid, lat_grid = np.meshgrid(longitude, np.linspace(-90, 90, var_data.shape[1]))

    # Plot the initial frame
    im = ax.pcolormesh(lon_grid, lat_grid, var_data[0], cmap='coolwarm', vmin=250, vmax=300, transform=ccrs.PlateCarree())

    ax.add_feature(coastline)
    ax.set_global()
    ax.gridlines()

    cbar = plt.colorbar(im, label='Temperature (K)', aspect=40, shrink=0.65)
    title = ax.set_title(f'Temperature - Frame 0')

    def update(frame):
        im.set_array(var_data[frame].ravel())  # Use flattened array
        title.set_text(f'Temperature - Frame {frame}')
        return [im, title]

    animation = FuncAnimation(fig, update, frames=range(var_data.shape[0]), interval=200)

    plt.close()  # Prevents displaying the initial plot

    animation_html = animation.to_jshtml()
    autoplay_html = animation_html.replace('controls>', 'controls autoplay>')

    return HTML(autoplay_html)

In [ ]:
# Example usage
var_data = get_variable_data(file_path, 't2m')
visualize_temperature_animation(var_data)

## visualize ECMWF ERA5 wind ( M/s)

In [ ]:

def visualize_wind_animation(var_data, projection='3d'):
    var_data = np.flip(var_data, axis=1)
    central_longitude = 130
    central_latitude = 10.0
    longitude = np.linspace(0, 360, var_data.shape[2])

    if projection == '3d':
        fig = plt.figure(figsize=(10, 6))
        ax = plt.axes(projection=ccrs.Orthographic(central_longitude=central_longitude, central_latitude=central_latitude))
    elif projection == '2d':
        fig = plt.figure(figsize=(10, 5))
        ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=central_longitude))
 
    else:
        raise ValueError("Invalid projection value. Choose either '2d' or '3d'.")

    coastline = cfeature.COASTLINE.with_scale('110m')

    # Create grid coordinates for pcolormesh
    lon_grid, lat_grid = np.meshgrid(longitude, np.linspace(-90, 90, var_data.shape[1]))

    # Plot the initial frame
    im = ax.pcolormesh(lon_grid, lat_grid, var_data[0],  vmin=-20, vmax=20,  transform=ccrs.PlateCarree())

    ax.add_feature(coastline)
    ax.set_global()
    ax.gridlines()

    cbar = plt.colorbar(im, label='wind (m/s)', aspect=40, shrink=0.65)
    title = ax.set_title(f'Temperature - Frame 0')

    def update(frame):
        im.set_array(var_data[frame].ravel())  # Use flattened array
        title.set_text(f'wind - Frame {frame}')
        return [im, title]

    animation = FuncAnimation(fig, update, frames=range(var_data.shape[0]), interval=200)

    plt.close()  # Prevents displaying the initial plot

    animation_html = animation.to_jshtml()
    autoplay_html = animation_html.replace('controls>', 'controls autoplay>')

    return HTML(autoplay_html)



In [ ]:
# Example usage
var_data = get_variable_data(file_path, 'u10')
visualize_wind_animation(var_data)

## visualize mean pressure level(hPa)

In [ ]:

def visualize_pressure_animation(var_data, projection='3d'):
    var_data = np.flip(var_data, axis=1)
    central_longitude = 130
    central_latitude = 10.0
    longitude = np.linspace(0, 360, var_data.shape[2])

    if projection == '3d':
        fig = plt.figure(figsize=(10, 6))
        ax = plt.axes(projection=ccrs.Orthographic(central_longitude=central_longitude, central_latitude=central_latitude))
    elif projection == '2d':
        fig = plt.figure(figsize=(10, 5))
        ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=central_longitude))
 
    else:
        raise ValueError("Invalid projection value. Choose either '2d' or '3d'.")

    coastline = cfeature.COASTLINE.with_scale('110m')

    # Create grid coordinates for pcolormesh
    lon_grid, lat_grid = np.meshgrid(longitude, np.linspace(-90, 90, var_data.shape[1]))

    # Plot the initial frame
    im = ax.pcolormesh(lon_grid, lat_grid, var_data[0]  , vmin=97_000, vmax=100_000, cmap='hot', transform=ccrs.PlateCarree())

    ax.add_feature(coastline)
    ax.set_global()
    ax.gridlines()

    cbar = plt.colorbar(im, label='pressure (hPa)', aspect=40, shrink=0.65)
    title = ax.set_title(f'MSL - Frame 0')

    def update(frame):
        im.set_array(var_data[frame].ravel())  # Use flattened array
        title.set_text(f'pressure - Frame {frame}')
        return [im, title]

    animation = FuncAnimation(fig, update, frames=range(var_data.shape[0]), interval=200)

    plt.close()  # Prevents displaying the initial plot

    animation_html = animation.to_jshtml()
    autoplay_html = animation_html.replace('controls>', 'controls autoplay>')

    return HTML(autoplay_html)

def visualize_pressure_animation_eastasia(var_data, projection='3d'):
    var_data = np.flip(var_data, axis=1)
    central_longitude = 130
    central_latitude = 10.0
    longitude = np.linspace(0, 360, var_data.shape[2])

    if projection == '3d':
        fig = plt.figure(figsize=(10, 6))
        ax = plt.axes(projection=ccrs.Orthographic(central_longitude=central_longitude, central_latitude=central_latitude))
    elif projection == '2d':
        fig = plt.figure(figsize=(10, 5))
        ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=central_longitude))
    else:
        raise ValueError("Invalid projection value. Choose either '2d' or '3d'.")

    coastline = cfeature.COASTLINE.with_scale('110m')

    # Create grid coordinates for pcolormesh
    lon_grid, lat_grid = np.meshgrid(longitude, np.linspace(-90, 90, var_data.shape[1]))

    # Plot the initial frame
    im = ax.pcolormesh(lon_grid, lat_grid, var_data[0], vmin=97_000, vmax=100_000, cmap='hot', transform=ccrs.PlateCarree())

    ax.add_feature(coastline)
    
    # Set the extent to focus on East Asia
    ax.set_extent([90, 160, -10, 60], crs=ccrs.PlateCarree())
    
    ax.gridlines()

    cbar = plt.colorbar(im, label='pressure (hPa)', aspect=40, shrink=0.65)
    title = ax.set_title(f'MSL - Frame 0')

    def update(frame):
        im.set_array(var_data[frame].ravel())  # Use flattened array
        title.set_text(f'pressure - Frame {frame}')
        return [im, title]

    animation = FuncAnimation(fig, update, frames=range(var_data.shape[0]), interval=200)

    plt.close()  # Prevents displaying the initial plot

    animation_html = animation.to_jshtml()
    autoplay_html = animation_html.replace('controls>', 'controls autoplay>')

    return HTML(autoplay_html)

In [ ]:
# Example usage
var_data = get_variable_data(file_path, 'msl')
visualize_pressure_animation(var_data)

## visualize with trajectory (sample baseline)

In [ ]:

def extract_typhoon_trajectory(var_data):
    import pandas as pd
    import numpy as np    
    trajectory_data = []

    # Flip data to have the correct lat-lon representation
    var_data = np.flip(var_data, axis=1)
    longitude = np.linspace(0, 360, var_data.shape[2])
    latitude = np.linspace(-90, 90, var_data.shape[1])
    lon_grid, lat_grid = np.meshgrid(longitude, latitude)

    # Mask latitudes outside the range 0 to +40
    lat_mask = (lat_grid >= 0) & (lat_grid <= 40)

    # Mask longitudes outside the range 100 to 180
    lon_mask = (lon_grid >= 100) & (lon_grid <= 180)

    # Apply the masks
    masked_data = var_data.copy()
    masked_data[:, ~lat_mask] = np.nan
    masked_data[:, ~lon_mask] = np.nan

    for idx, frame in enumerate(masked_data):
        min_pressure = np.nanmin(frame)  # Use nanmin to ignore NaN values
        typhoon_present = min_pressure < 99_000  # Define a suitable threshold

        # If typhoon is present, get the lat-lon coordinates
        if typhoon_present:
            typhoon_lat, typhoon_lon = lat_grid[frame == min_pressure][0], lon_grid[frame == min_pressure][0]
        else:
            typhoon_lat, typhoon_lon = np.nan, np.nan

        trajectory_data.append([idx, typhoon_present, typhoon_lon, typhoon_lat, min_pressure])

    # Convert list of lists to DataFrame
    df = pd.DataFrame(trajectory_data, columns=['idx_frame', 'typhoon_or_not', 'lon', 'lat', 'pressure'])

    return df

def visualize_pressure_animation_with_trajectory(var_data, df, projection='2d'):
    import pandas as pd
    import numpy as np    
    # Preparation for plotting
    var_data = np.flip(var_data, axis=1)
    central_longitude = 130
    central_latitude = 0
    longitude = np.linspace(0, 360, var_data.shape[2])

    if projection == '3d':
        fig = plt.figure(figsize=(10, 6))
        ax = plt.axes(projection=ccrs.Orthographic(central_longitude=central_longitude, central_latitude=central_latitude))
    elif projection == '2d':
        fig = plt.figure(figsize=(10, 5))
        ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=central_longitude))
    else:
        raise ValueError("Invalid projection value. Choose either '2d' or '3d'.")

    coastline = cfeature.COASTLINE.with_scale('110m')
    lon_grid, lat_grid = np.meshgrid(longitude, np.linspace(-90, 90, var_data.shape[1]))
    
    # Initial plot
    im = ax.pcolormesh(lon_grid, lat_grid, var_data[0], vmin=97_000, vmax=100_000,  cmap='hot', transform=ccrs.PlateCarree())
    ax.add_feature(coastline)
    ax.set_extent([100, 150, -5, 50], crs=ccrs.PlateCarree())
    ax.gridlines()
    cbar = plt.colorbar(im, label='pressure (hPa)', aspect=40, shrink=0.65)
    title = ax.set_title(f'MSL - Frame 0')
    
    # Typhoon trajectory line
    traj_line, = ax.plot([], [], color='black', transform=ccrs.PlateCarree())

    # Update function for animation
    def update(frame):
        im.set_array(var_data[frame].ravel())  # Use flattened array
        title.set_text(f'Pressure - Frame {frame}')

        # Update typhoon trajectory
        traj_df = df[df['idx_frame'] <= frame]
        traj_line.set_data(traj_df['lon'], traj_df['lat'])

        return [im, title, traj_line]

    animation = FuncAnimation(fig, update, frames=range(var_data.shape[0]), interval=200)
    plt.close()  # Prevents displaying the initial plot
    animation_html = animation.to_jshtml()
    autoplay_html = animation_html.replace('controls>', 'controls autoplay>')

    return HTML(autoplay_html)

In [ ]:
# Example usage
var_data = get_variable_data(file_path, 'msl')
df = extract_typhoon_trajectory(var_data)
visualize_pressure_animation_with_trajectory(var_data, df, projection='3d')

* baseline code track the lowest track so it have problem for multiple typhoons